# **Imports**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 44.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

import pickle
import matplotlib.pyplot as plt

import json
import csv
from pprint import pprint
import random
import numpy as np
import pandas as pd

import gensim.downloader
from gensim.models import KeyedVectors

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tqdm.notebook import tqdm

from pathlib import Path
from typing import Any, Tuple
from typing import Dict, Iterator, List, Union, Optional
import os

import gensim.downloader
from gensim.models import KeyedVectors

import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

from copy import deepcopy

from typing import Dict

from transformers import AutoTokenizer, BertTokenizer
from transformers import RobertaTokenizer, RobertaModel
from transformers import BertModel
from torchmetrics import Accuracy
from torch.optim import Adam

SEED = 1453

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


import pickle


import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torchmetrics

import torch.nn.functional as F

SEED: int = 42

pl.seed_everything(SEED)
import os
import re
import wandb
from pytorch_lightning.loggers import WandbLogger

from sklearn.metrics import classification_report

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk

from bs4 import BeautifulSoup
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## **Device**

In [ ]:
! nvidia-smi
device='cuda'

Fri Dec 29 13:33:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# device = 'cpu'

# data and preprocessing

well-known languages data

In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/NLP/extra/archive/dataset.csv"
normal_df = pd.read_csv(file_path)

normal_df.tail()

,Text,language
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese
21999,aprilie sonda spațială messenger a nasa și-a ...,Romanian


In [ ]:
bi_path = "/content/drive/MyDrive/Colab Notebooks/NLP/extra/archive/bible_languages.csv"
bible_df = pd.read_csv(bi_path)

bible_df = bible_df.sample(frac=1, random_state=42).reset_index(drop=True) #mix the elements

bible_df.head()

,Text,language
0,Tɨ ɨɨch je Jezabel nmöy tiempo co ixyipy yjoth...,Coatlan_Mixe
1,No ndab be men lea ta ndun Dios wa ta ulea nak...,Zapotec_San_Vicente_Coatlan
2,Waꞌa cheere Siimoni Peeroa ki nuꞌuti iyoriwapa...,Huarijio
3,x xo xt Ez He xm Jiwatjanithenij quꞌ niteꞌ...,Maka
4,t Natosian galevis ta sutuai luke q evi anatun...,Maskelyenes


## preprocessing

for the normal languages dataset

In [ ]:
def preprocess_data(text):
        # Remove HTML tags if present
    if "<" in text:
        text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove URL addresses
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove numbers
    text = re.sub(r'\d', '', text)

    # Remove parenthesis
    text = e.sub(r'\([^)]*\)', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()


    return text

In [ ]:
normal_df['Text'] = normal_df['Text'].apply(preprocess_data)

# Stampa le prime righe del DataFrame
print(normal_df.head())

                                                Text  language
0  klement gottwaldi surnukeha palsameeriti ning ...  Estonian
1  sebes joseph pereira thomas på eng the jesuits...   Swedish
2  ถนนเจรญกรง อกษรโรมน thanon charoen krung เรมตง...      Thai
3  வசகபபடடனம தமழசசஙகதத இநதப பததரகவசகபபடடன ஆசரயர ச...     Tamil
4  de spons behoort tot het geslacht haliclona en...     Dutch


for the bible dataset

In [ ]:
def rimuovi_parentesi(frase):
  #remove parenthesis
    return re.sub(r'\([^)]*\)', '', frase)

def rimuovi_numeri(frase):
  #remove numbers
    return re.sub(r'\d', '', frase)

def rimuovi_punteggiatura(testo):
  #remove punctuation
    return re.sub(r'[^\w\s]', '', testo)

In [ ]:
for i,frasi in enumerate(lista_di_frasi):

    frasi = get_frasi(lista_path[i])
    frasi = [frase.replace('\\v', '').replace('\\p','').replace('\\c', '').replace('\\q', '').replace('\\s', '').replace('\n', '').replace('\\f', '').replace('\\', '').replace('\\r', '') for frase in frasi]
    frasi = [rimuovi_numeri(frase) for frase in frasi]

    frasi = [rimuovi_parentesi(frase) for frase in frasi]

    frasi = [rimuovi_numeri(frase) for frase in frasi]

    frasi = [rimuovi_punteggiatura(frase) for frase in frasi]

    frasi = [frase.strip() for frase in frasi]

    frasi = frasi[5:]

    lista_di_frasi[i] = frasi

concatenation of dataframes

In [ ]:
df= pd.concat([normal_df, bible_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df

,Text,language
0,Lamtua Allah kit Aman na kon koma le halin nam...,Helong
1,jvk hall หองประชมขนาดยกษใหญ จผคนไดราวๆ คน มกใช...,Thai
2,그러나 하이잘 전투를 겪으며 스랄과 뜻을 같이하여 교류를 통해 호드를 아제로스의 일...,Korean
3,ووقت ل دخل للبيت سأل تلاميذو قاللن على أيش كان...,North_Mesopotamian_Arabic
4,Dedeng Ama Lamtua hutun soleng Un atulin Yahud...,Helong
...,...,...
46995,په لمريز کال د ادبياتو په رشته کې د لا لوړو زد...,Pushto
46996,Jè xita ra isìchjeén Nainá nga kiìchja ngajola...,Mazatec_San_Antonio
46997,Náaꞌa chóniwenai néeni sánaca narríshibia wayá...,Achagua
46998,پژواک د يوې ځانگړې پروژې په توگه د اساسي قانون...,Pushto


In [ ]:
#check if there are null values in the dataframe
null_values = df['Text'].isnull().any()

if null_values:
  print('There are null values in the column')
else:
  print('There are NOT null values in the dataframe')

There are NOT null values in the dataframe


## embeddings

in order to get the embeddings for every text I used a Doc2Vec model tha

In [ ]:
def get_embed(data):
  tagged_data = []
  for i,doc in enumerate(data):
    if isinstance(doc, float): print(doc)
    t = TaggedDocument(words=word_tokenize(doc.lower()),tags=[str(i)])
    tagged_data.append(t)

  # train the Doc2vec model
  model = Doc2Vec(vector_size=200,
                  min_count=2, epochs=10)

  model.build_vocab(tagged_data)
  model.train(tagged_data,
              total_examples=model.corpus_count,
              epochs=model.epochs)

  document_vectors = [model.infer_vector(
      word_tokenize(doc.lower())) for doc in data]


  return document_vectors

In [ ]:
t_embeddings = get_embed(df['Text'])


In [ ]:
t_embeddings[0].shape



(200,)

In [ ]:
df['embed_text'] = None

In [ ]:
df

,Text,language,embed_text
0,Lamtua Allah kit Aman na kon koma le halin nam...,Helong,None
1,jvk hall หองประชมขนาดยกษใหญ จผคนไดราวๆ คน มกใช...,Thai,None
2,그러나 하이잘 전투를 겪으며 스랄과 뜻을 같이하여 교류를 통해 호드를 아제로스의 일...,Korean,None
3,ووقت ل دخل للبيت سأل تلاميذو قاللن على أيش كان...,North_Mesopotamian_Arabic,None
4,Dedeng Ama Lamtua hutun soleng Un atulin Yahud...,Helong,None
...,...,...,...
46995,په لمريز کال د ادبياتو په رشته کې د لا لوړو زد...,Pushto,None
46996,Jè xita ra isìchjeén Nainá nga kiìchja ngajola...,Mazatec_San_Antonio,None
46997,Náaꞌa chóniwenai néeni sánaca narríshibia wayá...,Achagua,None
46998,پژواک د يوې ځانگړې پروژې په توگه د اساسي قانون...,Pushto,None


In [ ]:
len_df = int(df.shape[0])
for i in range(0,len_df):
  df.at[i,'embed_text'] = t_embeddings[i]


In [ ]:
df

,Text,language,embed_text
0,Lamtua Allah kit Aman na kon koma le halin nam...,Helong,"[0.029233547, 0.06535929, 0.032447107, -0.0156..."
1,jvk hall หองประชมขนาดยกษใหญ จผคนไดราวๆ คน มกใช...,Thai,"[-0.013957353, 0.027701112, 0.039279137, 0.040..."
2,그러나 하이잘 전투를 겪으며 스랄과 뜻을 같이하여 교류를 통해 호드를 아제로스의 일...,Korean,"[0.015088395, -0.16568483, -0.041877586, 0.230..."
3,ووقت ل دخل للبيت سأل تلاميذو قاللن على أيش كان...,North_Mesopotamian_Arabic,"[0.034551475, -0.06301372, -0.038499724, -0.00..."
4,Dedeng Ama Lamtua hutun soleng Un atulin Yahud...,Helong,"[0.03940029, -0.011168163, -0.021673605, -0.02..."
...,...,...,...
46995,په لمريز کال د ادبياتو په رشته کې د لا لوړو زد...,Pushto,"[-0.06007804, -0.033750463, -0.059150256, 0.13..."
46996,Jè xita ra isìchjeén Nainá nga kiìchja ngajola...,Mazatec_San_Antonio,"[0.015216741, 0.022256702, 0.04114828, -0.0227..."
46997,Náaꞌa chóniwenai néeni sánaca narríshibia wayá...,Achagua,"[0.01600722, -0.010104973, -0.012952803, 0.022..."
46998,پژواک د يوې ځانگړې پروژې په توگه د اساسي قانون...,Pushto,"[0.0079968395, -0.0538512, -0.036314595, 0.034..."


In [ ]:
emb_size = df['embed_text'][0].shape[0]
emb_size

200

In [ ]:
#check if there are null values in the dataframe
null_values = df['Text'].isnull().any()

if null_values:
  print('There are null values in the column')
else:
  print('There are NOT null values in the dataframe')

There are NOT null values in the dataframe


## dataframe division

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Calcola le dimensioni dei set di dati
train_size = int(train_ratio * len(df))
val_size = int(val_ratio * len(df))

# Suddividi il DataFrame
train_df = df[:train_size]
val_df = df[train_size:(train_size + val_size)]
test_df = df[(train_size + val_size):]

val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

len(train_df),len(val_df),len(test_df)

(28200, 9400, 9400)

In [ ]:
languages=list(df['language'].unique())
num_languages = len(languages)
languages,num_languages
vocab_languages = dict()
for lan in languages:
  if lan not in vocab_languages:
    vocab_languages[lan]=len(vocab_languages)
vocab_languages, num_languages

({'Pushto': 0,
  'Macushi': 1,
  'Korean': 2,
  'Persian': 3,
  'Helong': 4,
  'Spanish': 5,
  'Chol-Tumbala': 6,
  'Mazatec_Ixcatlan': 7,
  'Estonian': 8,
  'Angal_Heneng': 9,
  'Urdu': 10,
  'Chamacoco': 11,
  'Zapotec_San_Vicente_Coatlan': 12,
  'Arabic': 13,
  'Indonesian': 14,
  'Bimin': 15,
  'Mazatec_San_Jeronimo': 16,
  'Hindi': 17,
  'Swedish': 18,
  'Dutch': 19,
  'Mbya_Guarani': 20,
  'Tamil': 21,
  'Coatlan_Mixe': 22,
  'Portugese': 23,
  'Romanian': 24,
  'Thai': 25,
  'Latin': 26,
  'French': 27,
  'Plapo_Krumen': 28,
  'Mazatec_San_Antonio': 29,
  'Popoloca_San_Juan_Atzingo': 30,
  'English': 31,
  'Eastern_Kanjobal': 32,
  'Japanese': 33,
  'Metlatonoc_Mixtec': 34,
  'Achagua': 35,
  'Huehuetla_Tepehua': 36,
  'Maka': 37,
  'Huarijio': 38,
  'Tanimuca': 39,
  'Maskelyenes': 40,
  'Kapingamarangi': 41,
  'Borong': 42,
  'Turkish': 43,
  'North_Mesopotamian_Arabic': 44,
  'Chinese': 45,
  'Russian': 46},
 47)

# savings and loadings

In [ ]:
df_file = open("/content/drive/MyDrive/Colab Notebooks Eren/df_mio.pkl","wb")
pickle.dump(df,df_file)
df_file.close()


In [ ]:
per_df=open("/content/drive/MyDrive/Colab Notebooks Eren/df_mio.pkl","rb")
df =pickle.load(per_df)


In [ ]:
#creation of vocabularies that encode the languages
languages=list(df['language'].unique())
num_languages = len(languages)
languages,num_languages
vocab_languages = dict()
for lan in languages:
  if lan not in vocab_languages:
    vocab_languages[lan]=len(vocab_languages)
vocab_languages, num_languages

({'Pushto': 0,
  'Macushi': 1,
  'Korean': 2,
  'Persian': 3,
  'Helong': 4,
  'Spanish': 5,
  'Chol-Tumbala': 6,
  'Mazatec_Ixcatlan': 7,
  'Estonian': 8,
  'Angal_Heneng': 9,
  'Urdu': 10,
  'Chamacoco': 11,
  'Zapotec_San_Vicente_Coatlan': 12,
  'Arabic': 13,
  'Indonesian': 14,
  'Bimin': 15,
  'Mazatec_San_Jeronimo': 16,
  'Hindi': 17,
  'Swedish': 18,
  'Dutch': 19,
  'Mbya_Guarani': 20,
  'Tamil': 21,
  'Coatlan_Mixe': 22,
  'Portugese': 23,
  'Romanian': 24,
  'Thai': 25,
  'Latin': 26,
  'French': 27,
  'Plapo_Krumen': 28,
  'Mazatec_San_Antonio': 29,
  'Popoloca_San_Juan_Atzingo': 30,
  'English': 31,
  'Eastern_Kanjobal': 32,
  'Japanese': 33,
  'Metlatonoc_Mixtec': 34,
  'Achagua': 35,
  'Huehuetla_Tepehua': 36,
  'Maka': 37,
  'Huarijio': 38,
  'Tanimuca': 39,
  'Maskelyenes': 40,
  'Kapingamarangi': 41,
  'Borong': 42,
  'Turkish': 43,
  'North_Mesopotamian_Arabic': 44,
  'Chinese': 45,
  'Russian': 46},
 47)

In [ ]:
emb_size = df['embed_text'][0].shape[0]
emb_size

200

# dataset

In [ ]:
class LanDataset(Dataset):
  def __init__(self,

               df,
               vocab_languages):


    self.df = df
    self.vocab_languages= vocab_languages
    self.data = []
    for index, row in df.iterrows():
      text = row['Text']
      label = row['language']
      embed_text = row['embed_text']
      int_label = self.vocab_languages[label]

      item = {
          'text' : text.strip(),
          'embed_text' : list(embed_text),
          'label' : int(int_label),
          'str_label' : label
      }
      self.data.append(item)


  def __len__(self):
    return len(self.data)


  def __getitem__(self,index):
    item = self.data[index]

    return item

In [ ]:
prova = LanDataset(val_df,vocab_languages)

In [ ]:
prova.__getitem__(5)

{'text': 'Ngata paan Néná kuí boá xi kjindibáni ndiakajmii kó kuí xí tsjaá kjoamindyion xtaa kui ngasondiee',
 'embed_text': [-0.0027825588,
  -0.021993542,
  -0.008569635,
  0.011998151,
  0.023252103,
  0.010089947,
  -0.005893268,
  0.06053465,
  -0.01616629,
  0.03023413,
  -0.019684123,
  -0.00701916,
  0.0076704975,
  -0.0007147591,
  0.009248596,
  -0.026336363,
  -0.0032693723,
  0.0053122495,
  0.007676303,
  -0.039663684,
  0.01090563,
  -0.022071859,
  0.0073335418,
  -0.00059793814,
  0.001072992,
  -0.024577657,
  -0.012862296,
  -0.010254525,
  -0.038404383,
  -0.0016037488,
  0.02390588,
  -0.019289395,
  0.004175977,
  -0.012916431,
  -0.007835544,
  0.0063041593,
  -0.004819127,
  -0.0022493307,
  0.010267181,
  -0.024564348,
  -0.020737559,
  -0.0013049484,
  -0.009889797,
  0.014011955,
  0.022440704,
  -0.008450147,
  0.0012012598,
  -0.004484669,
  0.02171191,
  0.032579664,
  0.018485347,
  -0.0042786435,
  -0.00043677248,
  -0.0032109027,
  0.014991716,
  -0.0051

In [ ]:
vocab_languages

{'Pushto': 0,
 'Macushi': 1,
 'Korean': 2,
 'Persian': 3,
 'Helong': 4,
 'Spanish': 5,
 'Chol-Tumbala': 6,
 'Mazatec_Ixcatlan': 7,
 'Estonian': 8,
 'Angal_Heneng': 9,
 'Urdu': 10,
 'Chamacoco': 11,
 'Zapotec_San_Vicente_Coatlan': 12,
 'Arabic': 13,
 'Indonesian': 14,
 'Bimin': 15,
 'Mazatec_San_Jeronimo': 16,
 'Hindi': 17,
 'Swedish': 18,
 'Dutch': 19,
 'Mbya_Guarani': 20,
 'Tamil': 21,
 'Coatlan_Mixe': 22,
 'Portugese': 23,
 'Romanian': 24,
 'Thai': 25,
 'Latin': 26,
 'French': 27,
 'Plapo_Krumen': 28,
 'Mazatec_San_Antonio': 29,
 'Popoloca_San_Juan_Atzingo': 30,
 'English': 31,
 'Eastern_Kanjobal': 32,
 'Japanese': 33,
 'Metlatonoc_Mixtec': 34,
 'Achagua': 35,
 'Huehuetla_Tepehua': 36,
 'Maka': 37,
 'Huarijio': 38,
 'Tanimuca': 39,
 'Maskelyenes': 40,
 'Kapingamarangi': 41,
 'Borong': 42,
 'Turkish': 43,
 'North_Mesopotamian_Arabic': 44,
 'Chinese': 45,
 'Russian': 46}

# datamodule

In [ ]:
class LanDatamodule(pl.LightningDataModule):



    """
    """

    def __init__(
        self,
        batch_size : int
        ) -> None:
        super().__init__()


        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None) -> None:
        if stage == 'fit':
            self.train_dataset = LanDataset(train_df,vocab_languages)
            self.validation_dataset = LanDataset(val_df,vocab_languages)
        elif stage == 'test':
            self.test_dataset = LanDataset(test_df,vocab_languages)

    def train_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True, num_workers=2, collate_fn = self.prepare_batch)

    def val_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.validation_dataset, batch_size=self.batch_size, shuffle=False, drop_last=True, num_workers = 2, collate_fn = self.prepare_batch)

    def test_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, drop_last=True, num_workers = 2, collate_fn = self.prepare_batch)

    def prepare_batch(self,batch):

      """
      the collate_fn function, prepares the batch by giving a dictionary that will be the input of the forward function of the model
      """

      prepared_batch = {}

      prepared_batch['embed_text'] = [item['embed_text'] for item in batch]

      prepared_batch['labels'] = [item['label'] for item in batch]

      return prepared_batch

# model

In [ ]:


class LSTModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_rate, batch_size):
        super(LSTModel, self).__init__()
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_rate, batch_first=True, bidirectional = True)

        self.fc = nn.Linear(hidden_size*2, output_size)

        self.loss_function = nn.CrossEntropyLoss()

        self.accuracy = torchmetrics.Accuracy(task = 'multiclass', num_classes=output_size)

        self.preds = []
        self.targets = []

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)

        return output

    def training_step(self, batch, batch_idx):
        embeddings= torch.tensor(batch['embed_text']).to(device)#.to(self.device)
        labels =torch.tensor(batch['labels']).to(device)

        predictions = self(embeddings)
        loss = self.loss_function(predictions, labels)
        accuracy = self.accuracy(predictions, labels)
        self.log('train_loss', loss, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        self.log('train_acc', accuracy, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        return loss

    def validation_step(self, batch, batch_idx):

        embeddings= torch.tensor(batch['embed_text']).to(device)#.to(self.device)
        labels =torch.tensor(batch['labels']).to(device)

        predictions = self(embeddings)
        loss = self.loss_function(predictions, labels)
        accuracy = self.accuracy(predictions, labels)
        self.log('val_loss', loss, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        self.log('val_acc', accuracy, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        # return loss

    def test_step(self, batch, batch_idx):

        embeddings= torch.tensor(batch['embed_text']).to(device)#.to(self.device)
        labels =torch.tensor(batch['labels']).to(device)

        predictions = self(embeddings)

        self.preds += list(predictions)
        self.targets += list(labels)

        loss = self.loss_function(predictions, labels)
        accuracy = self.accuracy(predictions, labels)
        self.log('test_loss', loss, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        self.log('test_acc', accuracy, on_step=False, on_epoch=True, logger=True, batch_size=self.batch_size)
        # return loss

    def configure_optimizers(self):
        # Puoi personalizzare l'ottimizzatore e i suoi parametri qui
        return optim.Adam(self.parameters(), lr=0.001)


    def get_preds_targets(self):
      return self.preds, self.targets

# setup

wandb

In [ ]:
wandb.init(project='extra_lan_rec', dir='/content/drive/MyDrive/Colab Notebooks Eren')

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_acc,▅▅▃▅▃▇▆▁▅█
train_loss,█▅▄▃▃▂▂▁▂▁
trainer/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
val_acc,▇▂▆█▁▂▆▆▂▄
val_loss,▄█▄▄▁▃▃▄▃▂
epoch,9
train_acc,0.02234
train_loss,3.85067
trainer/global_step,4399
val_acc,0.02023


In [ ]:
wandb_logger = WandbLogger(
    project = 'extra_lan_rec',
    save_dir = '/content/drive/MyDrive/Colab Notebooks Eren',
    log_model = True,
    log_every_n_epochs = 1
)

In [ ]:

early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=10,
    verbose=True,
    mode='max',
)


In [ ]:
trainer = pl.Trainer(
    max_epochs=150,
    callbacks=early_stopping,
    logger = wandb_logger

)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
b_s = 64
mymodel = LSTModel(
    input_size=emb_size,
    hidden_size=64,
    output_size=len(vocab_languages),
    num_layers=2,
    dropout_rate=0.3,
    batch_size = b_s
    )

lan_dm= LanDatamodule(
    b_s
  )

In [ ]:
device = 'cuda'

# Fit and Test

In [ ]:
trainer.fit(model = mymodel, datamodule = lan_dm)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type               | Params
-----------------------------------------------------
0 | lstm          | LSTM               | 235 K 
1 | fc            | Linear             | 6.1 K 
2 | loss_function | CrossEntropyLoss   | 0     
3 | accuracy      | MulticlassAccuracy | 0     
-----------------------------------------------------
241 K     Trainable params
0         Non-trainable params
241 K     Total params
0.966     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_acc improved. New best score: 0.590


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_acc did not improve in the last 10 records. Best score: 0.888. Signaling Trainer to stop.


In [ ]:
trainer.test(model = mymodel, datamodule = lan_dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8816352486610413     │
│         test_loss         │    0.4112287163734436     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4112287163734436, 'test_acc': 0.8816352486610413}]

# Evaluation

In [ ]:
preds,targets = mymodel.get_preds_targets()
preds = [pred.argmax() for pred in preds]
preds_cpu = [pred.cpu().numpy()for pred in preds]
targets_cpu = [target.cpu().numpy() for target in targets]
int_vocab = {value: key for key, value in vocab_languages.items()}
class_labels = [int_vocab[i] for i in range(len(int_vocab))]
report = classification_report(targets_cpu, preds_cpu, target_names=class_labels)

print(report)

                             precision    recall  f1-score   support

                     Pushto       0.92      0.92      0.92       196
                    Macushi       0.91      0.89      0.90       219
                     Korean       0.95      0.98      0.97       186
                    Persian       0.95      0.96      0.96       194
                     Helong       0.88      0.81      0.84       201
                    Spanish       0.97      0.89      0.93       213
               Chol-Tumbala       0.80      0.88      0.84       182
           Mazatec_Ixcatlan       0.87      0.86      0.86       204
                   Estonian       0.94      0.93      0.94       181
               Angal_Heneng       0.87      0.82      0.85       199
                       Urdu       0.91      0.88      0.89       171
                  Chamacoco       0.89      0.90      0.90       183
Zapotec_San_Vicente_Coatlan       0.84      0.83      0.84       223
                     Arabic      

In [ ]:
%wandb